In [ ]:
!unzip archive\ \(12\).zip

Archive:  archive (12).zip
  inflating: Amazon_Policy_Self.csv  


In [ ]:
%%capture
!pip install -Uqqq pip --progress-bar off
!pip install -qqq bitsandbytes==0.41.3
!pip install -qqq torch--2.0.1 --progress-bar off
!pip install -qqq -U git+https://github.com/huggingface/transformers.git@e03a9cc --progress-bar off
!pip install -qqq -U git+https://github.com/huggingface/peft.git@42a184f --progress-bar off
!pip install -qqq -U git+https://github.com/huggingface/accelerate.git@c9fbb71 --progress-bar off
!pip install -qqq datasets==2.12.0 --progress-bar off
!pip install -qqq loralib==0.1.1 --progress-bar off
!pip install einops

In [ ]:
import os
# from pprint import pprint
# import json
import bitsandbytes as bnb
import pandas as pd
import torch
import torch.nn as nn
import transformers
from datasets import load_dataset
from huggingface_hub import notebook_login
from peft import (
    LoraConfig,
    PeftConfig,
    get_peft_model,
    prepare_model_for_kbit_training,
)
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
)

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
notebook_login()

In [ ]:
#Load Falcon Model Tokenizer
MODEL_NAME = "OpenAssistant/falcon-7b-sft-mix-2000"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    load_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

model =AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config,
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

config.json:   0%|          | 0.00/1.06k [00:00<?, ?B/s]

configuration_RW.py:   0%|          | 0.00/2.61k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/OpenAssistant/falcon-7b-sft-mix-2000:
- configuration_RW.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modelling_RW.py:   0%|          | 0.00/47.6k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/OpenAssistant/falcon-7b-sft-mix-2000:
- modelling_RW.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.74M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/450 [00:00<?, ?B/s]

In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [ ]:
config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 4718592 || all params: 3613536128 || trainable%: 0.13058101075667453


In [ ]:
prompt = """
<|prompter|>What is return policy ?  <|endoftext|><|assistant|>
""".strip()

In [ ]:
generation_config = model.generation_config
generation_config.max_new_tokens = 216
generation_config.temperature = 1
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

In [ ]:
generation_config

GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 1,
  "eos_token_id": 11,
  "max_new_tokens": 216,
  "pad_token_id": 11,
  "top_p": 0.7,
  "transformers_version": "4.30.0.dev0"
}

In [ ]:
%%time
device = "cuda:0"

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.inference_mode():
    outputs = model.generate(
      input_ids=encoding.input_ids,
      attention_mask=encoding.attention_mask,
      generation_config=generation_config,
  )
print(tokenizer.decode(outputs[0],skip_special_tokens=True))

What is return policy?  Return policy is a policy that outlines the terms and conditions under which a customer can return a product they have purchased. It typically includes information such as the time frame in which the product can be returned, the conditions under which the product must be returned (e.g. unopened, unused, in original packaging), and the process for initiating a return. Return policies can vary widely depending on the type of product and the seller, and they may be subject to change. It is important for customers to carefully review return policies before making a purchase to ensure they understand the terms and conditions under which they can return a product.
CPU times: user 1min 58s, sys: 331 ms, total: 1min 58s
Wall time: 2min 1s


In [ ]:
def generate_prompt(data_point):
    return f"""
<|prompter|> {data_point["Question"]}
<|endoftext|><|assistant|> {data_point["Answer"]}
  """.strip()

def generate_and_tokenize_prompt(data_point):
    full_prompt = generate_prompt(data_point)
    print(full_prompt)
    tokenized_full_prompt = tokenizer(full_prompt,padding=True, truncation=True)
    return tokenized_full_prompt

In [ ]:
import pandas as pd
df=pd.read_csv("/content/Amazon_Policy_Self.csv")

In [ ]:
from datasets import Dataset
dataset = Dataset.from_pandas(df)

In [ ]:
dataset = dataset.shuffle().map(generate_and_tokenize_prompt)

In [ ]:
OUTPUT_DIR = "/content/experiement"

In [ ]:
training_args = transformers.TrainingArguments(
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=10,
    learning_rate=2e-4,
    fp16=True,
    save_total_limit=4,
    logging_steps=2,
    output_dir=OUTPUT_DIR,
    max_steps=500,
    optim="paged_adamw_8bit",
    lr_scheduler_type = 'cosine',
    warmup_ratio = 0.05,
    report_to = 'tensorboard'
)

trainer = transformers.Trainer(
    model=model,
    train_dataset=dataset,
    args=training_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

In [ ]:
model.save_pretrained("trained-model")

In [ ]:
from peft import *

In [ ]:
config = PeftConfig.from_pretrained("/content/trained-model")

In [ ]:
config

PeftConfig(peft_type='LORA', base_model_name_or_path='OpenAssistant/falcon-7b-sft-mix-2000', revision=None, task_type='CAUSAL_LM', inference_mode=True)

In [ ]:
peft_model_id = "harsh1304/OpenAssistant_falcon-7b-sft-mix-2000_500steps_10Epochs"

config = PeftConfig.from_pretrained("/content/trained-model")



In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    return_dict=True,
    quantization_config=bnb_config,
    #device_map="cuda",
    trust_remote_code=True,
)

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
tokenizer.pad_token = tokenizer.eos_token


model = PeftModel.from_pretrained(model, peft_model_id)

In [ ]:
#import torch
#device="cuda" if torch.cuda.is_available() else "cpu"
#model.to(device)

In [ ]:
def generate_response(question: str) -> str:
    prompt = f"""
    <|prompter|> {question} <|endoftext|><|assistant|>
    """.strip()
    DEVICE = "cuda"
#     <|prompter|>What is return policy ?  <|endoftext|><|assistant|>
    encoding = tokenizer(prompt, return_tensors="pt").to(DEVICE)
    with torch.inference_mode():
        outputs = model.generate(
            input_ids=encoding.input_ids,
            attention_mask=encoding.attention_mask,
            generation_config=generation_config,
        )
    response = tokenizer.decode(outputs[0],skip_special_tokens=True)

#     assistant_start = '<assistant>:'
#     response_start = response.find(assistant_start)
#     return response[response_start + len(assistant_start):].strip()
    return response

In [ ]:
%%time
prompt = "what is general return policy of Amazon?"
result=generate_response(prompt)
result

CPU times: user 2min 22s, sys: 157 ms, total: 2min 22s
Wall time: 2min 23s


' what is general return policy of Amazon?  Products can be returned within the applicable return window on Amazon.in if they are physically damaged, have missing components or accessories, are defective, or differ from their description. Products returned within this window will be refunded. Products returned within this window due to being different from their description on Amazon.in are not eligible for a refund. Products returned due to being damaged, missing components or accessories, being defective, or differing from their description on Amazon.in within the applicable return window will result in a refund. Products returned outside of the applicable return window on Amazon.in will not be refunded. Amazon reserves the right to inspect the product to determine whether it is eligible for a refund before processing the request.'

In [ ]:
%%time
prompt = "How to cancel the order which was ordered before?"
result=generate_response(prompt)

CPU times: user 4min 25s, sys: 260 ms, total: 4min 25s
Wall time: 4min 34s


In [ ]:
result

' How to cancel the order which was ordered before?  Go to the "Your Orders" section and choose the option to "Request cancellation." Proceed by following the instructions provided. If you\'ve already made the payment, the item(s) will be returned to us for a refund. In case of payment-related issues, we will contact you to arrange the cancellation of the order. Should we be unable to offer the same product(s) in such a case, we will provide a refund. The outcome will be determined after we\'ve examined the product(s) and communicated with you. Please note that we hold the right to cancel the order if it turns out to be fraudulent. In such a situation, we will communicate with you and provide a chance to resolve the issue before proceeding with the cancellation. We will preserve the right to cancel the order in such a scenario. We will not be liable for any products or services procured utilizing the unauthorized access. We reserve the right to cancel the order in such a situation. We 

In [ ]:
%%time
prompt = "How to cancel the order which are shipped?"
result=generate_response(prompt)

CPU times: user 4min 29s, sys: 287 ms, total: 4min 29s
Wall time: 4min 31s


In [ ]:
result

' How to cancel the order which are shipped?  Access the section titled "Your Orders" and choose the option to "Request cancellation." Continue with the instructions provided. If you\'ve already made the payment, the item(s) will be returned to us for a refund. In case of payment being refused, the order will be canceled, with no further action taken. To ensure that your request is processed, make sure you\'ve followed the instructions correctly. Should we be unable to fulfill the request, we will provide a refund for the items involved. Note that we retain the right to cancel the order in case it turns out that it was fraudulent. Be aware that we might communicate with you to gather more information about the reason for the cancellation before we proceed. Should you disagree with the outcome of the cancellation, you have the right to escalate the matter to a third-party dispute resolution provider such as Resolve.io. We do not hold the right to cancel orders that are already shipped. 